In [32]:
from scipy.optimize import root
from scipy.integrate import odeint
from scipy import *
import matplotlib.pyplot as plt

In [104]:
### Four state serca model class
class serca:
    
    ## Must be in same order as dXdt return
    label = ['x0', 'x1', 'x2', 'y0', 'y1', 'y2', 'ca_e']
    nVar = len(label)
    
    vol_er = (3.9*0.1*0.1) # all units in um
    vol_cyt = (4.0*0.5*0.5-vol_er)
    vol_tot = vol_cyt + vol_er
    Pt=15e-6
    gamma=vol_er/vol_cyt
    c=100e-9
    
    cai = 100.0e-9
    cae = 0
    nserca = 8678.0
    N_avo=6.022e23
    initial = array([0.75554529, 0.18054556, 0.010757505, 0.015718635, 0.026363904, 0.011069105,1])
    initial = initial*nserca/(vol_er/1e18)/N_avo
    initial[6] = cae
    f=1
    
    
    #eq_value=[Pt/4,Pt/4,gamma*Pt/4,gamma*Pt/4,250e-6]
    
    ## Parameters
    p_orig = {
    'kx0_x1':2*1.0e8,
    'kx1_x2':1.0e8,
    'kx1_x0':83.666,
    'kx2_x1':2*83.666,
    'kx2_y2':0.6,
    'ky2_x2':0.097,
    'ky2_y1':2*30.015,
    'ky1_y0':30.015,
    'ky1_y2':1.0e5,
    'ky0_y1':2*1.0e5,
    'ky0_x0':0.4,
    'kx0_y0':1.20e-3
    }
    p=p_orig
    
    ## Get initial values for the system
    def __init__(self, X0=initial, T=arange(0.0, 1e-2, 1e-5)):
        self.T = T
        self.name = self.__class__.__name__
        if self.nVar != len(X0):
            print 'ERROR:', self.nVar, 'initial values required for:', self.name
        else: 
            self.X0 = X0
            
    
    def dXdt(self,X,t):
        x0, x1, x2, y0, y1, y2, ca_e = X 
        
        for var in self.p.keys():
            exec(var+'='+str(self.p[var]))
            
        dx0 = x0*(-kx0_x1*self.cai-kx0_y0)+x1*kx1_x0+y0*ky0_x0
        dx1 = x1*(-kx1_x2*self.cai-kx1_x0)+x0*self.cai*kx0_x1+x2*kx2_x1
        dx2 = x2*(-kx2_y2-kx2_x1)+x1*self.cai*kx1_x2+y2*ky2_x2
        dy0 = y0*(-ky0_y1*ca_e-ky0_x0)+y1*ky1_y0+x0*kx0_y0
        dy1 = y1*(-ky1_y2*ca_e-ky1_y0)+y0*ca_e*ky0_y1+y2*ky2_y1
        dy2 = y2*(-ky2_x2-ky2_y1)+y1*ca_e*ky1_y2+x2*kx2_y2
        dcae = -ca_e*(y1*ky1_y2 + y0*ky0_y1) + (y1*ky1_y0 + y2*ky2_y1)
        return [dx0, dx1, dx2, dy0, dy1, dy2, dcae]
    
    def update_p(self):
        for k in ['ky2_y1','kx1_x0','kx2_x1']:
            self.p[k]=self.f*self.p_orig[k]
            #print self.p
    
    def func_tbs(self,f,value):
        self.f=f
        self.update_p()
        X=odeint(self.dXdt,self.initial,t=arange(0.0, 1e-2, 1e-5))
        return X[-1][6]-value

In [107]:
S=serca()
S.func_tbs(1.0,2.5e-4)

0.00035197852111922836

In [101]:
sol=root(S.func_tbs,5.0,args=(5.0e-4,),method='broyden1',tol=1e-3)

TypeError: can't multiply sequence by non-int of type 'numpy.float64'